In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer

import torch
import torch.nn as nn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.get_device_name()

'GeForce GTX 1050'

In [2]:
breast_features, malignant = load_breast_cancer(return_X_y=True, as_frame=True)
breast_features

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [3]:
normed_breast_features = StandardScaler().fit_transform(breast_features)
X_train, X_test, y_train, y_test = train_test_split(normed_breast_features, malignant, test_size=.2, random_state=8)

In [4]:
"""CCEA w/ 2 populations to optimize ANGN wts & N-G parameters

ANGN wts are codified u/ a variable-length real-valued chromosome whose size
corresponds to # of connection wts among artificial neurons.

N-G parameters deals w/ a mixed coding scheme to represent 5 params that model
neuron-glia interactions.
"""
# Chromosome 1: Connection weights: real valued genes
# TODO

# Chromosome 2: N-G parameters: integer coded genes
# k >= mu
k = torch.range(3, 10)  # Number of iterations
mu = torch.range(2, 8)  # Number of activations

# a < b
a = torch.range(10, 90, 10)  # Percentage of increment over ANN weights
b = torch.range(20, 100, 10)  # Percentage of decrement over ANN weights

algo_type = torch.range(1, 7)

In [5]:
torch.set_grad_enabled(False)

def u(neuron_output):  # indicates if neuron has ﬁred or not.
    return 1 if neuron_output > 0 else -1


class Astrocyte:
    def __init__(self, k, mu, a, b):
        self.y = torch.empty(k)
        self.mu, self.a, self.b = mu, a, b

    def attenuatedEffectOfAstrocyte(self, t):
        mu = self.mu
        r = u(self.y[t])

        if t > 0:
            prev_effect = self.attenuatedEffectOfAstrocyte(t - 1)
            if -mu < prev_effect < mu:
                r += prev_effect
            elif prev_effect in (-mu, mu):
                r = prev_effect

        return r

    def z(self):
        r = self.attenuatedEffectOfAstrocyte(len(self.y)-1)
        mu = self.mu

        if r == mu:
            return self.a
        if r == -mu:
            return -self.b
        return 0


class ThreeLayerANGN(nn.Module):
    def __init__(self, n_inputs, hidden_layer_units, n_outputs, k, mu, a, b):
        super().__init__()
        self.fc1 = nn.Linear(n_inputs, hidden_layer_units, bias=True)  # fully connected
        self.astrocytes1 = [Astrocyte(k, mu, a, b) for _ in range(hidden_layer_units)]
        self.fc2 = nn.Linear(hidden_layer_units, n_outputs, bias=True)  # fully connected
        self.astrocytes2 = [Astrocyte(k, mu, a, b) for _ in range(n_outputs)]
        self.k = k
        self.k_itr = 0

    def forward(self, inputs):
        n_inputs = len(inputs)
        out = torch.empty(n_inputs, dtype=torch.bool)

        for itr in range(n_inputs):  # row-by-row processing
            out[itr] = self.forward_single_pass(inputs[itr]) > .5

        return out

    def forward_single_pass(self, x):
        astrocytes1, astrocytes2 = self.astrocytes1, self.astrocytes2
        n_astrocytes1, n_astrocytes2 = len(astrocytes1), len(astrocytes2)
        fc1, fc2 = self.fc1, self.fc2
        k = self.k_itr

        y = torch.tanh(fc1(x))
        for itr in range(n_astrocytes1):
            astrocytes1[itr].y[k] = y[itr]

        y = torch.sigmoid(fc2(y))
        for itr in range(n_astrocytes2):
            astrocytes2[itr].y[k] = y[itr]

        self.k_itr += 1
        if self.k_itr == self.k:
            for itr in range(n_astrocytes1):
                multiplier = astrocytes1[itr].z() / 100
                fc1.weight[itr] = nn.Parameter(fc1.weight[itr] + fc1.weight[itr] * multiplier)

            for itr in range(n_astrocytes2):
                multiplier = astrocytes2[itr].z() / 100
                fc2.weight[itr] = nn.Parameter(fc2.weight[itr] + fc2.weight[itr] * multiplier)

            self.k_itr = 0

        return y

In [6]:
nw = ThreeLayerANGN(n_inputs=9, hidden_layer_units=7, n_outputs=1, k=3, mu=2, a=20, b=10)
nw(torch.Tensor(X_train[:, :9]))
torch.true_divide((nw(torch.Tensor(X_test[:, :9])) == torch.Tensor(y_test)).sum(), y_test.shape[0])

tensor(0.7719)

In [7]:
# 3 attributes describing interaction mechanism among populations in CEAs
INTERACTION = {
    'SAMPLE_SIZE': [20],  # collaborators employed to eval individual
    'SELECTIVE_BIAS': ['random', 'fitness'],  # mthd for selecting collaborator/competitors
    'CREDIT_ASSIGNMENT': ['min', 'max', 'mean']  # mthd for computing fitness
}